# Coursera IBM capstone project -  The Battle of Neighborhoods

<p> project title: </p>

<h2>Finding a proper location for opening a new business in the fututre that is suitable during the ‘covid - 19’ pandemic in london, United Kingdom (U.K) using Data science and analytical tools</h2>

<p> Author: Raihan Insan Habibi</p>

<b>1. Introduction / business problems</b>

In the middle of march, the UK was struck by a deadly virus called the coronavirus also known by its scientific name 'covid-19'. and the rate of the virus kept increasing dramatically till February 1, 2021, and till that day the rate of the corona virus started to decrease gradually. Right now, social distancing has changed various methods for many companies. With time, things are gently progressing back to normal, people who want to open up a business in the future will have to have to find a suitable location that will be effective for the industry business, even during the 'covid-19' pandemic. Finding a location that is habitable requires various factors that data scientists need to consider and process, as well as all of the access to data information around the neighbourhood. In this report, I will be describing a simple analysis of data from the whole city of london, United Kingdom (U.K) to find a suitable location to open up a new business.

<b>2. Data</b>

(all data resources will be here)

In [104]:
#!pip install geopy
#!pip install folium
#!pip install bs4

You should consider upgrading via the 'c:\users\mathl\anaconda3\python.exe -m pip install --upgrade pip' command.


In [123]:
import numpy as np
import pandas as pd
import requests
from geopy.geocoders import Nominatim
import folium
from pandas.io.json import json_normalize
import bs4
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
print("libaries imported")

libaries imported


In [124]:
CLIENT_ID = 'PT4SWYOA3Q4IJPMPYFKE4MYTKG3YBFPBHPUB01C5MVC0BCHE' 
CLIENT_SECRET = 'ZYIMZWRM5PDVXZGFOJXKRZ2H423NPVFUWLBQXHDYMJSMDGEL' 
VERSION = '20180604'
LIMIT = 30

address = 'London, Greater London, United Kingdom'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
coords = (latitude, longitude)
print(coords)
search_query = 'apartment'
radius = 2000
LIMIT = 50

(51.5073219, -0.1276474)


In [125]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
url

'https://api.foursquare.com/v2/venues/search?client_id=PT4SWYOA3Q4IJPMPYFKE4MYTKG3YBFPBHPUB01C5MVC0BCHE&client_secret=ZYIMZWRM5PDVXZGFOJXKRZ2H423NPVFUWLBQXHDYMJSMDGEL&ll=51.5073219,-0.1276474&v=20180604&query=apartment&radius=2000&limit=50'

In [126]:
venues = results['response']['venues']
df = json_normalize(venues)

C:\Users\raihan\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


In [127]:
#cleaning the data
df = df.drop(["id", "categories","location.country","referralId", "hasPerk",'location.labeledLatLngs', 'location.cc','location.formattedAddress' ], axis = 1)
df = df.rename(columns = {"name":"Residents",
                          "location.lat":"latitude", 
                          'location.lng':"longitude",
                          'location.distance':"distance",
                          'location.address':"address", 
                          "location.city":"city",
                          'location.crossStreet':"street name",
                           'location.postalCode':"postcode",
                          'location.neighborhood':"neighborhood",
                           'location.state':"state"
                         })
new_distance = []
df["distance"] = df["distance"].astype(str)
df["distance"] = df["distance"].tolist()
df = df.head(10)

In [128]:
london_map = folium.Map(coords, zoom_start = 10)
lat_list = df["latitude"].tolist()
lng_list = df["longitude"].tolist()
for x in range(len(lat_list)):
    folium.Marker([lat_list[x], lng_list[x]]).add_to(london_map)

In [129]:
london_map

In Here im getting all the location for all residentail area because a good business and rate of virus depends on the population of people in london 

In [130]:
columns = []
list_inside = []
my_url = "https://en.wikipedia.org/wiki/List_of_areas_of_London"
uClient = uReq(my_url)
page_html = uClient.read()
uClient.close()
page_soup = soup(page_html, "html.parser")
big_container = page_soup.findAll("div", {"id":"mw-content-text"}) 
table_container = big_container[0].findAll("table")  
in_table = table_container[1].findAll("tbody")
row_table = in_table[0].findAll("tr")
each_row = in_table[0].findAll("th")
dataframe = {}




In [131]:
for x in each_row:
    columns.append(x.text.rstrip("\n"))
for row_container in row_table:
    row_list = row_container.findAll("td")
    for x in row_list:
        list_inside.append(x.text.rstrip("\n"))
        

    

In [134]:


def table_maker(headers, columns_list):
    no_cols = len(headers)
    df_list = []
    starting_point = -1 * no_cols
    test_run = len(columns_list) // no_cols - 1
    end_point = len(columns_list) // no_cols
    variable_list = []
    var = {}

    for p in range(no_cols):
        f_var = int(p)
        variable_list.append(f_var)

    for x in variable_list:
        var[x] = []

    try:
        while test_run >= 0:
            starting_point += no_cols
            end_point += no_cols
            df_list.append(columns_list[starting_point:end_point])
            test_run -= 1

    except:
        print("not divisible")

    for x1 in range(no_cols):
        for y in df_list:
            var[x1].append(y[x1])

    df = pd.DataFrame(var)
    renamed = {}
    for p in range(no_cols):
        renamed[p] = columns[p]
        dataframe_final = df.rename(columns=renamed)
        
    
    #print(dataframe_final.columns.tolist())
    return dataframe_final
    
    


In [135]:
dataframe_final = table_maker(columns, list_inside)

# cleaning the dataframe
dataframe_final = dataframe_final.rename(columns = {"Location":"Part in London", 
                                                    "Postcode district":"Postcode",
                                                   "Post town": "town"})

dataframe_final = dataframe_final.drop(columns=['Dial\xa0code', 'OS grid ref'], axis = 1)
dataframe_final = dataframe_final[dataframe_final.town == "LONDON"]


dataframe_final = dataframe_final.head(20)
dataframe_final

,Part in London,London borough,town,Postcode district
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4"
6,Aldgate,City[10],LONDON,EC3
7,Aldwych,Westminster[10],LONDON,WC2
9,Anerley,Bromley[11],LONDON,SE20
10,Angel,Islington[8],LONDON,"EC1, N1"
12,Archway,Islington[12],LONDON,N19
15,Arnos Grove,Enfield[12],LONDON,"N11, N14"
16,Balham,Wandsworth[13],LONDON,SW12
17,Bankside,Southwark[14],LONDON,SE1


In [136]:
london_part = dataframe_final["Part in London"].tolist()
long_list = []
lat_list = []
attempt = 0
for part in london_part:
    address = part + ", London"
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    long_list.append(longitude)
    lat_list.append(latitude )
    attempt += 1

In [137]:
dataframe_final["latitude"] = lat_list
dataframe_final["longitude"] = long_list

dataframe_final

,Part in London,London borough,town,Postcode district,latitude,longitude
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,51.487621,0.114050
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",51.508140,-0.273261
6,Aldgate,City[10],LONDON,EC3,51.514248,-0.075719
7,Aldwych,Westminster[10],LONDON,WC2,51.512200,-0.118896
9,Anerley,Bromley[11],LONDON,SE20,51.407599,-0.061939
10,Angel,Islington[8],LONDON,"EC1, N1",51.531842,-0.105714
12,Archway,Islington[12],LONDON,N19,51.565437,-0.134998
15,Arnos Grove,Enfield[12],LONDON,"N11, N14",51.616402,-0.133287
16,Balham,Wandsworth[13],LONDON,SW12,51.445645,-0.150364
17,Bankside,Southwark[14],LONDON,SE1,51.507499,-0.099302


In [138]:
london_corona_test = pd.read_csv("london_corona_test.csv")

In [139]:
london_corona_test

,area_name,area_code,date,new_cases,total_cases
0,Barking and Dagenham,E09000002,2020-02-11,0,0
1,Barnet,E09000003,2020-02-11,0,0
2,Bexley,E09000004,2020-02-11,0,0
3,Brent,E09000005,2020-02-11,0,0
4,Bromley,E09000006,2020-02-11,0,0
...,...,...,...,...,...
12571,Sutton,E09000029,2021-03-09,3,15607
12572,Tower Hamlets,E09000030,2021-03-09,2,28222
12573,Waltham Forest,E09000031,2021-03-09,15,24466
12574,Wandsworth,E09000032,2021-03-09,1,21527


In [140]:
london_covid_19_df = london_corona_test.tail(10)
london_covid_19_df

,area_name,area_code,date,new_cases,total_cases
12566,Merton,E09000024,2021-03-09,1,15812
12567,Newham,E09000025,2021-03-09,4,34757
12568,Redbridge,E09000026,2021-03-09,5,32133
12569,Richmond upon Thames,E09000027,2021-03-09,4,10503
12570,Southwark,E09000028,2021-03-09,1,22063
12571,Sutton,E09000029,2021-03-09,3,15607
12572,Tower Hamlets,E09000030,2021-03-09,2,28222
12573,Waltham Forest,E09000031,2021-03-09,15,24466
12574,Wandsworth,E09000032,2021-03-09,1,21527
12575,Westminster,E09000033,2021-03-09,4,13683


In [141]:
#cleaning the data

london_covid_19_df = london_covid_19_df.rename(columns= {"area_name":"Part in London",
                                   "area_code":"area code",
                                   "new_cases": "new cases",
                                   "total_cases":"total cases"})
london_covid_19_df  = london_covid_19_df.drop(columns=["new cases", "area code"], axis= 1)


In [142]:
london_part = london_covid_19_df["Part in London"].tolist()
long_list2 = []
lat_list2 = []
attempt = 0
for part in london_part:
    address = part + ", London"
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    long_list2.append(longitude)
    lat_list2.append(latitude)
    attempt += 1

In [143]:
print(len(long_list2))
print(len(lat_list2))

10
10


In [144]:
london_covid_19_df["latitude"] = lat_list2
london_covid_19_df["longitude"] = long_list2

london_covid_19_df = london_covid_19_df.sort_values(by=["total cases"])
london_covid_19_df


,Part in London,date,total cases,latitude,longitude
12569,Richmond upon Thames,2021-03-09,10503,51.440553,-0.307639
12575,Westminster,2021-03-09,13683,51.500444,-0.126540
12571,Sutton,2021-03-09,15607,51.357464,-0.173627
12566,Merton,2021-03-09,15812,51.410870,-0.188097
12574,Wandsworth,2021-03-09,21527,51.457027,-0.193261
12570,Southwark,2021-03-09,22063,51.502922,-0.103458
12573,Waltham Forest,2021-03-09,24466,51.598169,-0.017837
12572,Tower Hamlets,2021-03-09,28222,51.525629,-0.033585
12568,Redbridge,2021-03-09,32133,51.576320,0.045410
12567,Newham,2021-03-09,34757,51.530000,0.029318


In [145]:
k = 22/6381872

In [146]:
print(k*10503 * 1000)

36.20661774476204


In [147]:
population_list = london_covid_19_df["total cases"].tolist()
population_list

[10503, 13683, 15607, 15812, 21527, 22063, 24466, 28222, 32133, 34757]

In [148]:
for t in range(len(lat_list2)):
    folium.Marker([lat_list2[t], long_list2[t]], icon=folium.Icon(color="red")).add_to(london_map)
    folium.CircleMarker([lat_list2[t], long_list2[t]], radius = population_list[t] * k * 1000  ).add_to(london_map)

In [149]:
london_map